# Hướng dẫn sử dụng hệ thống RAG UIT@PubHealthQA

Notebook này hướng dẫn cách sử dụng hệ thống RAG đã được tái cấu trúc. Hệ thống RAG cho phép tìm kiếm và truy xuất thông tin từ các văn bản pháp luật y tế.

## 1. Cài đặt thư viện cần thiết

Đảm bảo bạn đã cài đặt các thư viện cần thiết:

In [1]:
# Cài đặt các thư viện cần thiết (chỉ chạy một lần)
# !pip install -U langchain faiss-cpu langchain-huggingface sentence-transformers tqdm pypdf

## 2. Import các thư viện và module

In [2]:
import json
import time
import logging
from pathlib import Path

# Import các module từ thư mục src
import sys
sys.path.append('..')

from src.utils.logging_utils import setup_logging
from src.embed.faiss_manager import initialize_embedding_model, load_vector_db
from src.retriever.faiss_retriever import query_documents

# Thiết lập logging
log_file_path = Path("../outputs/logs/rag_usage_log.txt")
setup_logging(log_file_path=log_file_path, console_output=True)

2025-04-21 16:48:57,685 - INFO - Đã thiết lập cấu hình logging.
2025-04-21 16:48:57,687 - INFO - File log: ..\outputs\logs\rag_usage_log.txt
2025-04-21 16:48:57,687 - INFO - File log: ..\outputs\logs\rag_usage_log.txt


## 3. Cấu hình các tham số

In [3]:
# Đường dẫn đến vector database
PERSIST_DIRECTORY = Path("../data/gold/db_faiss_phapluat_yte_full_final")
# Model embedding sử dụng
MODEL_NAME = "bkai-foundation-models/vietnamese-bi-encoder"

print(f"Thư mục lưu Index FAISS: {PERSIST_DIRECTORY}")
print(f"Model Embedding: {MODEL_NAME}")

Thư mục lưu Index FAISS: ..\data\gold\db_faiss_phapluat_yte_full_final
Model Embedding: bkai-foundation-models/vietnamese-bi-encoder


## 4. Khởi tạo model embedding và tải vector database

In [4]:
# Khởi tạo model embedding
embeddings = initialize_embedding_model(model_name=MODEL_NAME)
if not embeddings:
    logging.error("Không thể khởi tạo model embedding. Hãy kiểm tra lại model_name hoặc cài đặt thư viện.")
    raise RuntimeError("Không thể khởi tạo model embedding.")

# Tải vector database
vectordb = load_vector_db(persist_directory=PERSIST_DIRECTORY, embeddings=embeddings)
if not vectordb:
    logging.error(f"Không thể tải vector database từ {PERSIST_DIRECTORY}. Kiểm tra lại đường dẫn hoặc chạy chunking.ipynb để tạo vector database.")
    raise RuntimeError("Không thể tải vector database.")

print(f"\nĐã tải vector database thành công. Số lượng vector: {vectordb.index.ntotal}")

2025-04-21 16:49:49,900 - WARNING - From c:\Users\phamd\AppData\Local\Programs\Python\Python312\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

2025-04-21 16:49:50,652 - INFO - PyTorch version 2.6.0.dev20241206+cu126 available.
2025-04-21 16:49:50,652 - INFO - TensorFlow version 2.18.0 available.
2025-04-21 16:49:50,652 - INFO - PyTorch version 2.6.0.dev20241206+cu126 available.
2025-04-21 16:49:50,652 - INFO - TensorFlow version 2.18.0 available.
2025-04-21 16:49:52,090 - INFO - Load pretrained SentenceTransformer: bkai-foundation-models/vietnamese-bi-encoder
2025-04-21 16:49:52,090 - INFO - Load pretrained SentenceTransformer: bkai-foundation-models/vietnamese-bi-encoder
2025-04-21 16:49:56,130 - INFO - Model Embedding 'bkai-foundation-models/vietnamese-bi-encoder' đã sẵn sàng.
2025-04-21 16:49:56,140 - INFO - Loading faiss with AVX2 support.
2025-04-21 


Đã tải vector database thành công. Số lượng vector: 15498


## 5. Thực hiện truy vấn cơ bản

In [5]:
def display_search_results(results, show_metadata=True):
    """Hiển thị kết quả tìm kiếm một cách rõ ràng."""
    if not results:
        print("Không tìm thấy kết quả nào.")
        return
    
    for i, result in enumerate(results):
        print(f"\n--- Kết quả {i+1} ---")
        
        # Kiểm tra xem result có phải là tuple (doc, score) không
        if isinstance(result, tuple) and len(result) == 2:
            doc, score = result
            print(f"Điểm số: {score:.4f}")
        else:
            doc = result
        
        # Hiển thị metadata nếu được yêu cầu
        if show_metadata and hasattr(doc, 'metadata') and doc.metadata:
            print("Metadata:")
            # Hiển thị các thông tin quan trọng
            doc_id = doc.metadata.get('document_id', 'N/A')
            doc_type = doc.metadata.get('document_type', 'N/A')
            effective_date = doc.metadata.get('effective_date', 'N/A')
            location = doc.metadata.get('location_detail', 'N/A')
            
            print(f"  - ID văn bản: {doc_id}")
            print(f"  - Loại văn bản: {doc_type}")
            print(f"  - Ngày hiệu lực: {effective_date}")
            print(f"  - Vị trí: {location}")
        
        # Hiển thị nội dung
        content = doc.page_content if hasattr(doc, 'page_content') else str(doc)
        print("\nNội dung:")
        print(content)
        print("-" * 80)

In [6]:
# Thử nghiệm truy vấn cơ bản
query = "Quy định về đăng ký thuốc mới"
k = 3  # Số lượng kết quả trả về

print(f"Câu hỏi: {query}")
print(f"Tìm kiếm {k} kết quả phù hợp nhất...")

start_time = time.time()
results = query_documents(vectordb, query, k=k)
end_time = time.time()

print(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")
display_search_results(results)

2025-04-21 16:49:56,876 - INFO - Đã tiền xử lý query: 'Quy định về đăng ký thuốc mới'


Câu hỏi: Quy định về đăng ký thuốc mới
Tìm kiếm 3 kết quả phù hợp nhất...


2025-04-21 16:49:58,070 - INFO - Truy vấn 'Quy định về đăng ký thuốc mới' hoàn tất sau 1.241 giây. Tìm thấy 3 kết quả.


Thời gian truy vấn: 1.250 giây

--- Kết quả 1 ---
Metadata:
  - ID văn bản: 08/2022/TT-BYT
  - Loại văn bản: Thông tư
  - Ngày hiệu lực: 2022-10-20
  - Vị trí: Chương I: QUY ĐỊNH CHUNG... -> Điều 9: Tiêu chí phân loại và các trường hợp công bố biệt dược gốc, sinh phẩm tham chiếu... -> Khoản 2 -> Điểm a

Nội dung:
- Thuốc được cấp giấy đăng ký lưu hành mới theo hình thức đăng ký lại quy định tại Thông tư số 44/2014/TT-BYT ngày 25 tháng 11 năm 2014 của Bộ trưởng Bộ Y tế quy định việc đăng ký thuốc (sau đây gọi tắt là Thông tư số 44/2014/TT-BYT) mà có cùng công thức bào chế, quy trình sản xuất, tiêu chuẩn chất lượng nguyên liệu, tiêu chuẩn chất lượng thuốc thành phẩm so với biệt dược gốc, sinh phẩm tham chiếu đã được công bố hoặc có thay đổi liên quan đến nội dung trên đã được Cục Quản lý Dược hoặc nước sở tại phê duyệt. Cơ sở đăng ký nộp hồ sơ cập nhật phân loại biệt dược gốc, sinh phẩm tham chiếu theo quy định tại Phụ lục II ban hành kèm theo Thông tư này; - Thuốc thay đổi cơ sở sản xuấ

## 6. Truy vấn với điểm số

In [7]:
# Truy vấn với điểm số
query = "Thẩm quyền thu hồi giấy chứng nhận"
k = 3

print(f"Câu hỏi: {query}")
print(f"Tìm kiếm {k} kết quả phù hợp nhất với điểm số...")

start_time = time.time()
results = query_documents(vectordb, query, k=k, with_score=True)
end_time = time.time()

print(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")
display_search_results(results)

2025-04-21 16:49:58,093 - INFO - Đã tiền xử lý query: 'Thẩm quyền thu hồi giấy chứng nhận'
2025-04-21 16:49:58,157 - INFO - Truy vấn 'Thẩm quyền thu hồi giấy chứng nhận' hoàn tất sau 0.064 giây. Tìm thấy 3 kết quả.
2025-04-21 16:49:58,157 - INFO - Truy vấn 'Thẩm quyền thu hồi giấy chứng nhận' hoàn tất sau 0.064 giây. Tìm thấy 3 kết quả.


Câu hỏi: Thẩm quyền thu hồi giấy chứng nhận
Tìm kiếm 3 kết quả phù hợp nhất với điểm số...
Thời gian truy vấn: 0.067 giây

--- Kết quả 1 ---
Điểm số: 25.9119
Metadata:
  - ID văn bản: 08/2025/TT-BYT
  - Loại văn bản: Thông tư
  - Ngày hiệu lực: 2025-03-07
  - Vị trí: Điều 7: Thẩm quyền, trình tự thu hồi giấy chứng nhận...

Nội dung:
Điều 7. Thẩm quyền, trình tự thu hồi giấy chứng nhận 1. Bộ Y tế (Cục An toàn thực phẩ m) ban hành văn bản thu hồi giấy chứng nhận và gửi cho tổ chức, cá nhân xuất khẩu đã được cấp giấy chứng nhận; đồng thời đăng tải thông tin trên trang thông tin điện tử của Cục và gửi thông báo đến cơ quan hải quan về việc giấy chứng nhận không còn giá trị hiệu lực; Giấy chứng nhận hết hiệu lực kể từ ngày cơ quan có thẩm quyền ban hành văn bản thu hồi. 2. Trong thời hạn 05 (nă m) ngày làm việc, tổ chức, cá nhân được cấp giấy chứng nhận phải nộp giấy chứng nhận đã cấp cho Bộ Y tế (Cục An toàn thực phẩ m).
---------------------------------------------------------------------

## 7. Truy vấn với MMR (Maximum Marginal Relevance)

In [8]:
# Truy vấn với MMR (đa dạng hóa kết quả)
query = "Nội dung giấy chứng nhận cần có thông tin gì?"
k = 4
fetch_k = 20  # Số lượng kết quả để chọn lọc

print(f"Câu hỏi: {query}")
print(f"Tìm kiếm {k} kết quả đa dạng nhất (MMR) từ {fetch_k} kết quả ban đầu...")

start_time = time.time()
results = query_documents(vectordb, query, k=k, fetch_k=fetch_k, use_mmr=True)
end_time = time.time()

print(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")
display_search_results(results)

2025-04-21 16:49:58,173 - INFO - Đã tiền xử lý query: 'Nội dung giấy chứng nhận cần có thông tin gì?'
2025-04-21 16:49:58,244 - INFO - Truy vấn 'Nội dung giấy chứng nhận cần có thông tin gì?' hoàn tất sau 0.071 giây. Tìm thấy 4 kết quả.
2025-04-21 16:49:58,244 - INFO - Truy vấn 'Nội dung giấy chứng nhận cần có thông tin gì?' hoàn tất sau 0.071 giây. Tìm thấy 4 kết quả.


Câu hỏi: Nội dung giấy chứng nhận cần có thông tin gì?
Tìm kiếm 4 kết quả đa dạng nhất (MMR) từ 20 kết quả ban đầu...
Thời gian truy vấn: 0.071 giây

--- Kết quả 1 ---
Metadata:
  - ID văn bản: 18/2019/TT-BYT
  - Loại văn bản: Thông tư
  - Ngày hiệu lực: 2019-07-17
  - Vị trí: Điều 4 -> Khoản 2

Nội dung:
2. Yêu cầu đối với nội dung của giấy chứng nhận, chứng nhận tương đương quy định tại các Điểm a, b và c Khoản 1 Điều này bao gồm các thông tin sau: a) Tên cơ quan, tổ chức có thẩm quyền cấp; b) Ngày cấp; c) Thời hạn hiệu lực (trong trường hợp giấy chứng nhận không ghi thời hạn hiệu lực thì phải có tài liệu chứng minh cơ sở sản xuất vẫn duy trì điều kiện an toàn thực phẩm: báo cáo đánh giá hoặc biên bản kiểm tra định kỳ); d) Họ tên, chữ ký của người cấp; đ) Tên, địa chỉ cơ sở được cấp; e) Phạm vi được chứng nhận; g) Dạng bào chế hoặc tên sản phẩm được chứng nhận.
--------------------------------------------------------------------------------

--- Kết quả 2 ---
Metadata:
  - ID văn bản

## 8. Giao diện truy vấn tương tác

In [9]:
def interactive_search():
    """Giao diện tìm kiếm tương tác."""
    print("\n=== GIAO DIỆN TÌM KIẾM VĂN BẢN PHÁP LUẬT Y TẾ ===\n")
    print("Nhập 'exit' để thoát.")
    
    while True:
        query = input("\nNhập câu hỏi của bạn: ")
        if query.lower() == 'exit':
            print("Tạm biệt!")
            break
            
        # Lấy tùy chọn
        try:
            k = int(input(f"Số lượng kết quả (mặc định: 3): ") or "3")
            use_mmr = input("Sử dụng MMR để đa dạng hóa kết quả? (y/n, mặc định: n): ").lower() == 'y'
            
            if use_mmr:
                fetch_k = int(input(f"fetch_k cho MMR (mặc định: {k*5}): ") or str(k*5))
            else:
                fetch_k = k*5
                
            show_score = input("Hiển thị điểm số? (y/n, mặc định: n): ").lower() == 'y'
            
            # Thực hiện truy vấn
            print(f"\nĐang tìm kiếm cho câu hỏi: '{query}'...")
            start_time = time.time()
            
            results = query_documents(
                vectordb, 
                query, 
                k=k, 
                fetch_k=fetch_k, 
                use_mmr=use_mmr, 
                with_score=show_score
            )
            
            end_time = time.time()
            print(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")
            
            # Hiển thị kết quả
            display_search_results(results)
            
        except Exception as e:
            print(f"Lỗi: {e}")
            logging.error(f"Lỗi khi thực hiện truy vấn: {e}", exc_info=True)

In [ ]:
# Chạy giao diện tìm kiếm tương tác
interactive_search()


=== GIAO DIỆN TÌM KIẾM VĂN BẢN PHÁP LUẬT Y TẾ ===

Nhập 'exit' để thoát.


2025-04-21 16:50:27,463 - INFO - Đã tiền xử lý query: 'bảo hiểm y tế'
2025-04-21 16:50:27,515 - INFO - Truy vấn 'bảo hiểm y tế' hoàn tất sau 0.053 giây. Tìm thấy 2 kết quả.
2025-04-21 16:50:27,515 - INFO - Truy vấn 'bảo hiểm y tế' hoàn tất sau 0.053 giây. Tìm thấy 2 kết quả.



Đang tìm kiếm cho câu hỏi: 'bảo hiểm y tế'...
Thời gian truy vấn: 0.055 giây

--- Kết quả 1 ---
Điểm số: 33.4590
Metadata:
  - ID văn bản: 30/2018/TT-BYT
  - Loại văn bản: Thông tư
  - Ngày hiệu lực: 2019-01-01
  - Vị trí: Điều 2: Cấu trúc danh mục thuốc và phân hạng sử dụng... -> Khoản 1 -> Điểm d

Nội dung:
- Đối với phòng khám tư nhân có ký hợp đồng khám bệnh, chữa bệnh bảo hiểm y tế nhưng chưa thực hiện phân tuyến chuyên môn kỹ thuật: Căn cứ năng lực chuyên môn, trang thiết bị y tế và danh mục dịch vụ kỹ thuật đã được cấp có thẩm quyền phê duyệt, Sở Y tế chủ trì, phối hợp với Bảo hiểm xã hội tỉnh để thống nhất, quyết định việc sử dụng thuốc của cơ sở khám bệnh, chữa bệnh này nhưng không quá phạm vi danh mục thuốc của bệnh viện hạng III; - Trường hợp cơ sở khám bệnh, chữa bệnh thực hiện được các dịch vụ kỹ thuật của tuyến cao hơn theo phân tuyến chuyên môn kỹ thuật thì được sử dụng các thuốc theo danh mục thuốc quy định đối với tuyến cao hơn, phù hợp với dịch vụ kỹ thuật đã được cấ

2025-04-21 16:50:58,738 - ERROR - Lỗi khi thực hiện truy vấn: invalid literal for int() with base 10: 'exit'
Traceback (most recent call last):
  File "C:\Users\phamd\AppData\Local\Temp\ipykernel_6744\3263814769.py", line 14, in interactive_search
    k = int(input(f"Số lượng kết quả (mặc định: 3): ") or "3")
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: 'exit'


Lỗi: invalid literal for int() with base 10: 'exit'


2025-04-21 16:51:04,263 - INFO - Đã tiền xử lý query: ''
2025-04-21 16:51:04,310 - INFO - Truy vấn '' hoàn tất sau 0.047 giây. Tìm thấy 3 kết quả.
2025-04-21 16:51:04,310 - INFO - Truy vấn '' hoàn tất sau 0.047 giây. Tìm thấy 3 kết quả.



Đang tìm kiếm cho câu hỏi: ''...
Thời gian truy vấn: 0.048 giây

--- Kết quả 1 ---
Metadata:
  - ID văn bản: 10/2018/TT-BYT
  - Loại văn bản: Thông tư
  - Ngày hiệu lực: 2018-06-18
  - Vị trí: Chương II: VỊ TRÍ, CHỨC NĂNG, NHIỆM VỤ,... -> Điều 8: Tiêu chí của thành viên Hội đồng...

Nội dung:
Điều 8. Tiêu chí của thành viên Hội đồng 1. Thành viên Hội đồng tư vấn cấp phép kinh doanh cho cơ sở sản xuất, xuất khẩu, nhập khẩu, kinh doanh dịch vụ bảo quản, bán buôn, bán lẻ đối với thuốc gây nghiện, thuốc hướng thần, thuốc tiền chất, nguyên liệu làm thuốc là dược chất gây nghiện, dược chất hướng thần, tiền chất dùng làm thuốc; cơ sở sản xuất thuốc dạng phối hợp có chứa dược chất gây nghiện, thuốc dạng phối hợp có chứa dược chất hướng thần, thuốc dạng phối hợp có chứa tiền chất phải đáp ứng đồng thời các tiêu chí sau: a) Có bằng tốt nghiệp đại học trở lên thuộc một trong các chuyên ngành y, dược, an ninh, cảnh sát.
-----------------------------------------------------------------------------

## 9. Kết hợp với LLM để xây dựng hệ thống RAG hoàn chỉnh

Để tạo một hệ thống RAG hoàn chỉnh, bạn có thể kết hợp kết quả truy xuất với một Large Language Model (LLM). Dưới đây là ví dụ cách tích hợp với LLM từ OpenAI hoặc mô hình mã nguồn mở.

In [ ]:
# Ví dụ tích hợp với LLM (yêu cầu cài đặt thêm thư viện)
# !pip install langchain-openai  # Cho OpenAI
# !pip install langchain-community  # Cho các mô hình mã nguồn mở

# Nhận xét các dòng sau nếu chưa cài đặt thư viện cần thiết
'''
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# Đặt API key (thay thế bằng API key của bạn hoặc sử dụng biến môi trường)
import os
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Khởi tạo LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Tạo retriever từ vector database
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# Tạo chuỗi RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True
)

# Sử dụng
query = "Quy định về đăng ký thuốc mới tại Việt Nam?"
result = qa_chain({"query": query})
print("Câu trả lời:")
print(result["result"])
'''